<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg4/meta-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Meta Learning
Using Omni glot dataset

In [ ]:
# ref: https://github.com/shashankhalo7/Omniglot_meta_learning/blob/master/osl.ipynb

In [1]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip

--2020-10-16 20:22:44--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip [following]
--2020-10-16 20:22:45--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9464212 (9.0M) [application/zip]
Saving to: ‘images_background.zip’

images_background.z 100%[===================>]   9.03M  51.0MB/s    in 0.2s    

2020-10-16 20:22:46 (51.0 MB/s) - ‘images_background.zip’ saved [946

In [2]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip

--2020-10-16 20:22:47--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip [following]
--2020-10-16 20:22:48--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6462886 (6.2M) [application/zip]
Saving to: ‘images_evaluation.zip’

images_evaluation.z 100%[===================>]   6.16M  --.-KB/s    in 0.1s    

2020-10-16 20:22:48 (47.6 MB/s) - ‘images_evaluation.zip’ saved [646

In [3]:
!unzip -q images_background.zip
!unzip -q images_evaluation.zip

In [4]:
!ls

images_background      images_evaluation      sample_data
images_background.zip  images_evaluation.zip


In [5]:
import sys
import numpy as np
import pandas as pd
from imageio import imread # from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

In [6]:
!mkdir data

In [7]:
train_folder = "images_background/"
val_folder = "images_evaluation/"
save_path = "data/"

In [12]:
def loadimages(path, n=0):
  '''
  Parameters:
  string:path -> Path of train or test directory

  Return Value:
  tuple:(x, y, language_dictionary)
  '''
  X=[]
  y=[]
  cat_dict = {}
  lang_dict = {}
  curr_y = n

  for alphabet in os.listdir(path):
    print("loading alphabet: " + alphabet)
    lang_dict[alphabet] = [curr_y, None]
    alphabet_path = os.path.join(path, alphabet)
    # Set each letter as its own column
    for letter in os.listdir(alphabet_path):
      cat_dict[curr_y] = (alphabet, letter)
      category_images=[]
      letter_path = os.path.join(alphabet_path, letter)
      # read images in current category
      for filename in os.listdir(letter_path):
        image_path = os.path.join(letter_path, filename)
        image = imread(image_path)
        category_images.append(image)
        y.append(curr_y)
      try:
        X.append(np.stack(category_images))
      # edge case - last one
      except ValueError as e:
        print(e)
        print("error - category_images:", category_images)
      curr_y += 1
      lang_dict[alphabet][1] = curr_y - 1
  y = np.vstack(y)
  X = np.stack(X)
  return X,y,lang_dict

In [13]:
X, y, c = loadimages(train_folder)

loading alphabet: Hebrew
loading alphabet: Greek
loading alphabet: Armenian
loading alphabet: Mkhedruli_(Georgian)
loading alphabet: Latin
loading alphabet: Burmese_(Myanmar)
loading alphabet: Tagalog
loading alphabet: Arcadian
loading alphabet: Futurama
loading alphabet: Anglo-Saxon_Futhorc
loading alphabet: Ojibwe_(Canadian_Aboriginal_Syllabics)
loading alphabet: Braille
loading alphabet: Alphabet_of_the_Magi
loading alphabet: Early_Aramaic
loading alphabet: N_Ko
loading alphabet: Blackfoot_(Canadian_Aboriginal_Syllabics)
loading alphabet: Malay_(Jawi_-_Arabic)
loading alphabet: Japanese_(katakana)
loading alphabet: Tifinagh
loading alphabet: Balinese
loading alphabet: Gujarati
loading alphabet: Asomtavruli_(Georgian)
loading alphabet: Bengali
loading alphabet: Inuktitut_(Canadian_Aboriginal_Syllabics)
loading alphabet: Grantha
loading alphabet: Syriac_(Estrangelo)
loading alphabet: Cyrillic
loading alphabet: Korean
loading alphabet: Japanese_(hiragana)
loading alphabet: Sanskrit


In [14]:
with open(os.path.join(save_path, "train.pickle"), "wb") as f:
  pickle.dump((X,c ), f)

In [16]:
Xval, yval, cval = loadimages(val_folder)

loading alphabet: Malayalam
loading alphabet: Gurmukhi
loading alphabet: Old_Church_Slavonic_(Cyrillic)
loading alphabet: Syriac_(Serto)
loading alphabet: Kannada
loading alphabet: ULOG
loading alphabet: Avesta
loading alphabet: Mongolian
loading alphabet: Glagolitic
loading alphabet: Manipuri
loading alphabet: Ge_ez
loading alphabet: Sylheti
loading alphabet: Atlantean
loading alphabet: Keble
loading alphabet: Angelic
loading alphabet: Tengwar
loading alphabet: Tibetan
loading alphabet: Aurek-Besh
loading alphabet: Oriya
loading alphabet: Atemayar_Qelisayer


In [17]:
with open(os.path.join(save_path, "val.pickly"), "wb") as f:
  pickle.dump((Xval, cval), f)

In [18]:
def initialize_weights(shape, name=None):
  '''
     Initialize CNN layer weights with mean as 0.0 and standard deviation 0f 0.01
  '''
  return np.random.normal(loc = 0.0, scale = 1e-2, size=shape)

In [ ]:
def initialize_bias(shape, name=None):
  '''

  '''
  return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)